In [1]:
docs = ["오늘 동물원에서 원숭이를 봤어", 
       "오늘 동물원에서 코끼리를 봤어 봤어",
       "동물원에서 원숭이에게 바나나를 줬어 바나나를"]

sklearn

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect=TfidfVectorizer() #사전
tfidf = tfidf_vect.fit_transform(docs)
tfidf

<3x8 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [5]:
type(tfidf)
#csr_matrix: 0 필요없는 부분 제외하고 보여줌

scipy.sparse.csr.csr_matrix

In [6]:
#한눈에 보기 위함
tfidf.todense()

matrix([[0.37311881, 0.        , 0.4804584 , 0.4804584 , 0.63174505,
         0.        , 0.        , 0.        ],
        [0.28680065, 0.        , 0.73861611, 0.36930805, 0.        ,
         0.        , 0.        , 0.48559571],
        [0.2344005 , 0.79374908, 0.        , 0.        , 0.        ,
         0.39687454, 0.39687454, 0.        ]])

In [7]:
tfidf_vect.get_feature_names_out()

array(['동물원에서', '바나나를', '봤어', '오늘', '원숭이를', '원숭이에게', '줬어', '코끼리를'],
      dtype=object)

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
BoW = count_vect.fit_transform(docs)
BoW.todense()
tfidf_trans = TfidfTransformer()
tfidf = tfidf_trans.fit_transform(BoW)
tfidf.todense()
#알고리즘 다르기 때문에 어제와 결과 다름; 사이킷런 결과 따르기

matrix([[0.37311881, 0.        , 0.4804584 , 0.4804584 , 0.63174505,
         0.        , 0.        , 0.        ],
        [0.28680065, 0.        , 0.73861611, 0.36930805, 0.        ,
         0.        , 0.        , 0.48559571],
        [0.2344005 , 0.79374908, 0.        , 0.        , 0.        ,
         0.39687454, 0.39687454, 0.        ]])

In [10]:
import pandas as pd

vocab = count_vect.get_feature_names_out()
pd.DataFrame(tfidf.todense(), columns=vocab)

,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
0,0.373119,0.000000,0.480458,0.480458,0.631745,0.000000,0.000000,0.000000
1,0.286801,0.000000,0.738616,0.369308,0.000000,0.000000,0.000000,0.485596
2,0.234400,0.793749,0.000000,0.000000,0.000000,0.396875,0.396875,0.000000


gensim

In [15]:
#gensim은 TDM, 사이킷런은 DTM기준
import gensim
from gensim import corpora
from gensim.models import TfidfModel

doc_ls = [doc.split() for doc in docs]
id2word = corpora.Dictionary(doc_ls) #젠심은 우선 딕셔너리로 만들어야 함
DTM = [id2word.doc2bow(doc) for doc in doc_ls] #doc2bow
model = TfidfModel(DTM)
tfidf = model[DTM]
tfidf[0] #첫번째 단어

[(1, 0.32718457421365993), (2, 0.32718457421365993), (3, 0.8865102981879297)]

In [16]:
for key in id2word.keys():
    print("{} : {}".format(key, id2word[key]))

0 : 동물원에서
1 : 봤어
2 : 오늘
3 : 원숭이를
4 : 코끼리를
5 : 바나나를
6 : 원숭이에게
7 : 줬어


In [17]:
from gensim.matutils import sparse2full

vocab = [id2word[key] for key in id2word.keys()]
DTM_matrix = [sparse2full(doc, len(vocab)).tolist() for doc in tfidf]
pd.DataFrame(DTM_matrix, columns=vocab)

,동물원에서,봤어,오늘,원숭이를,코끼리를,바나나를,원숭이에게,줬어
0,0.0,0.327185,0.327185,0.88651,0.000000,0.000000,0.000000,0.000000
1,0.0,0.569307,0.284654,0.00000,0.771272,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.00000,0.000000,0.816497,0.408248,0.408248


신문기사 크롤링 키워드 추출

In [78]:
from konlpy.tag import Mecab
# import nltk
# from nltk import bigrams,word_tokenize
# from nltk.util import ngrams

mecab = Mecab()

In [79]:
# bigram_list = bigrams(mecab.morphs(sentences[0]))
# bigram_list

<generator object bigrams at 0x0000022B9456B548>

In [153]:
import re
import requests
from bs4 import BeautifulSoup

강사님 풀이

In [187]:
#계속 반복되는 일이니까 함수 설정
import requests
from bs4 import BeautifulSoup

from konlpy.tag import Mecab
mecab = Mecab()

def get_news(url):
    headers = {'user-agent':'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    
    article=html.select_one("#articleBodyContents").text.strip()
    
    article = " ".join([token[0] for token in mecab.pos(article) if token[1] in ["NNG","NNP","VV"]]) 
    #list정보 하나씩 토큰으로 옴. 한 기사의 본문 내용 한번에 보여지도록 띄어쓰기로 join
    
    return article

urls=['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108',
'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790',
'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362',
'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191',
'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728']

docs=[] #명사,동사 뽑아낸 걸 기사당 하나의 문서로 만들기
for url in urls:
    docs.append(get_news(url)) #정규식 사용해서 자르기

In [188]:
#doc에 잘 들어갔는지 30번째까지만 체크
for doc in docs:
    print(doc[:30])

과기 정통부 유영민 장관 참석 기념행사 투입 여종 데이
한국전력공사 한국전력공사 제공 나주 연합뉴스 송 기자 
판교 자율 주행 실증 자율 주행 데이터 개방 사업 선정
통계청 텔레콤 모바일 빅 데이터 분석 들 회복세 코로나
금융 결제 원 신용 정보원 지정 비씨 카드 추가 포함 


In [189]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(max_df=0.85,max_features=10000)
#min_df:문서1%미만 나타나는 용어 제거
#max_df:너무 자주 나타나는 용어 제거; 문서 50%이상 나타나는 용어 무시
#max_feature:단어 사전에 들어가는 전체 단어의 수 제한

#단어 리스트 만들기
word_count_vector = count_vect.fit_transform(docs)

In [190]:
#unique한 단어들의 리스트
count_vect.vocabulary_

{'과기': 48,
 '정통부': 406,
 '유영민': 328,
 '장관': 375,
 '참석': 453,
 '기념행사': 77,
 '투입': 494,
 '여종': 292,
 '구축': 61,
 '민간': 162,
 '클라우드': 479,
 '외부': 315,
 '연계': 298,
 '체계': 460,
 '개방': 16,
 '강화': 14,
 '데일리': 124,
 '이재운': 353,
 '국가': 63,
 '차원': 449,
 '시대': 263,
 '산업': 213,
 '창출': 455,
 '기존': 85,
 '변화': 178,
 '이르': 344,
 '혁신': 539,
 '분야': 192,
 '경제': 33,
 '발전': 167,
 '정부': 404,
 '청사진': 457,
 '현실': 540,
 '구현': 62,
 '계획': 38,
 '과학': 49,
 '기술': 80,
 '정보': 402,
 '통신부': 491,
 '서울': 230,
 '중구': 428,
 '대한': 120,
 '상공': 214,
 '회의소': 559,
 '생태': 225,
 '조성': 416,
 '성장': 237,
 '기반': 79,
 '마련': 139,
 '플랫': 512,
 '출범식': 471,
 '행사': 530,
 '개최': 21,
 '노웅래': 99,
 '국회': 67,
 '방송': 170,
 '통신': 490,
 '위원회': 326,
 '위원장': 325,
 '참가': 451,
 '이미지': 346,
 '픽사': 515,
 '베이': 174,
 '이루': 343,
 '문재': 154,
 '핵심': 529,
 '요소': 317,
 '문재인': 155,
 '대통령': 119,
 '보호': 182,
 '보안': 180,
 '중요': 432,
 '강조': 13,
 '맥락': 148,
 '공공': 40,
 '협업': 544,
 '양질': 284,
 '생산': 224,
 '수집': 254,
 '분석': 191,
 '유통': 330,
 '역할': 295,
 '담당': 110,
 '기관

In [196]:
#데이터로 transformer에 학습만 시킴
tfidf_tranformer = TfidfTransformer()
tfidf_tranformer.fit(word_count_vector) #tfidf는 전체정보 필요하므로 전체 문장으로 훈련

TfidfTransformer()

In [197]:
#데이터 기준 sorting
#key: sorting 기준 설정
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1],x[0]), reverse=True)

#점수 기준 sorting
def extract_keywords(feature_name, sorted_keywords, n=5):
    return [(feature_name[idx], score) for idx,score in sorted_keywords[:n]]

In [198]:
#tfidf 높은 순서대로 sorting
feature_name = count_vect.get_feature_names_out()
tf_idf_vector = tfidf_tranformer.transform(count_vect.transform([docs[0]]))
sorted_keywords = sort_keywords(tf_idf_vector.tocoo())
keywords = extract_keywords(feature_name, sorted_keywords)
keywords

[('플랫', 0.2630245621515846),
 ('계획', 0.22544962470135824),
 ('정통부', 0.18787468725113188),
 ('과기', 0.18787468725113188),
 ('혁신', 0.15157611471898802)]

In [200]:
for i in range(len(docs)):
    tf_idf_vector = tfidf_transform.transform(count_vect.transform([docs[i]]))
    sorted_keywords = sort_keywords(tf_idf_vector.tocoo())
    keywords = extract_keywords(feature_name,sorted_keywords)
    print(f"문서 {i + 1}")
    print(keywords)
    print("\n")

문서 1
[('플랫', 0.2630245621515846), ('계획', 0.22544962470135824), ('정통부', 0.18787468725113188), ('과기', 0.18787468725113188), ('혁신', 0.15157611471898802)]


문서 2
[('한전', 0.3835353685325492), ('전력', 0.3835353685325492), ('대회', 0.2301212211195295), ('서비스', 0.17286170921899544), ('제공', 0.15411486374360572)]


문서 3
[('주행', 0.6610125681327058), ('자율', 0.6334703777938431), ('경기도', 0.22220865745662136), ('실증', 0.13771095169431372), ('개방', 0.11110432872831068)]


문서 4
[('인구', 0.4009591863854239), ('발생', 0.29860739716747037), ('이동', 0.2775871290360627), ('지역', 0.2737234474035145), ('통계청', 0.24674411469872237)]


문서 5
[('중계', 0.6921577195028837), ('기관', 0.3973252213196342), ('금융', 0.25528159538995704), ('인프라', 0.21753528327233487), ('중앙', 0.11865560905763721)]




내 풀이

In [146]:
url_list=['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108',
'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790',
'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362',
'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191',
'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728']
headers = {'user-agent':'Mozilla/5.0'}

article_list=[]

for i in range(len(url_list)):
    temp=[]
    response = requests.get(url_list[i],headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    article=html.select_one("#articleBodyContents").text.strip()
    temp.append(article)
    article_list.append(temp)
article_list

[['과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다. 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다. 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다. 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다.주요 활용 전략·사례를 보면 빅데이터 활용

In [149]:
article_list[0]

['과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다. 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다. 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다. 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다.주요 활용 전략·사례를 보면 빅데이터 활용을

In [150]:
clean_words=[]
docs=[]
for i in range(len(article_list)):
    for word in mecab.pos(article_list[i]):
        if word[1] in ['NNG','NNP','NNB','NNBC','VV']:
            clean_words.append(word[0])
    docs.append(" ".join(clean_words))
docs

AssertionError: phrase input should be string, not <class 'list'>

In [128]:
# from nltk import sent_tokenize

# sentences = sent_tokenize(article_list)
# sentences

TypeError: expected string or bytes-like object

In [125]:
clean_words=[]
docs=[]
for i in range(len(sentences)):
    for word in mecab.pos(sentences[i]):
        if word[1] in ['NNG','NNP','NNB','NNBC','VV']:
            clean_words.append(word[0])
docs.append(" ".join(clean_words))
docs

['금융 결제 원 신용 정보원 등 지정 비씨 카드 추가 포함 여부 논의 시행령 발표 후 추진 하반기 데이터 유통 판매 급물살 정부 금융 결제 원 신용 정보원 등 업 곳 데이터 중계 기관 선정 일 서울 중구 한국 신용 정보원 데이터 태 포스 데이터 전송 항목 표준 관련 실무 회의 진행 이동근 기자 정부 미래 신사업 마이 데이터 중계 기관 곳 최종 선정 데이터 중계 기관 기관 개인 금융 금융 데이터 번 통합 조회 때 데이터 사업자 금융 기관 연결 효율 데이터 수 있 허브 역할 담당 중계 기관 경우 기업 금융 기관 별도 계약 맺 데이터 연결 효율 발생 중계 기관 금융 기관 다수 핀테크 사업자 등 대 연결 데이터 유통 관리 중계 일종 정보 기술 인프라 운영 일 정부 금융 따르 금융 위원회 데이터 중계 기관 곳 최종 선정 인프라 개발 작업 착수 선정 중계 기관 금융 결제 원 신용 정보원 농협 중앙 수협 중앙 상호저축은행 중앙 신협 중앙 마을금고 중앙 코스콤 행정 정보 공유 센터 중계 기관 앞 업 유형 데이터 관리 유통 허브 역할 전담 이종 산업 간 데이터 결합 마이 데이터 산업 핵심 역할 담당 곳 외 카드 데이터 중계 기관 프로세싱 인프라 보유 비씨카드 중계 기관 포함 것 논의 진행 것 다수 데이터 개별 계약 관리 중계 기간 효율 데이터 관리 전략 금융 위 데이터 산업 활성 공청회 등 열 공청회 당시 중계 기관 선정 놓 의견 일부 기업 중계 기관 데이터 독점 수 있 우려 지속 제기 다수 금융 사 기업 데이터 산업 진흥 중계 기관 필요 입장 제시 코로나 여파 중계 기관 간 공식 회의 열리 인프라 개발 구축 작업 논의 시작 대형 인프라 설비 필요 때문 시스템 통합 업계 사업 기회 것 전망 개 기관 가운데 데이터 중계 핵심 역할 금융 결제 원 신용 정보원 전담 가능 일부 중계 기관 경우 인프라 개발 경험 전무 데이터 중계 수 있 시스템 자체 비씨카드 중계 기관 허용 논의 쟁점 업 대변 여신 금융 협회 대신 카드 정보 실제 프로세싱 수 있 비씨카드 중계 역할 의견 제기 내부 

In [112]:
arti = clean_words.join()
arti

AttributeError: 'list' object has no attribute 'join'

In [109]:
clean_ls=[clean_word.split(".") for clean_word in clean_words]
clean_ls

511

In [105]:
from collections import defaultdict
word2id = defaultdict(lambda : len(word2id))
[word2id[token] for clean_word in clean_ls for token in clean_word]
word2id

defaultdict(<function __main__.<lambda>()>,
            {'과기': 0,
             '정통부': 1,
             '일': 2,
             '유영민': 3,
             '장관': 4,
             '등': 5,
             '참석': 6,
             '기념행사': 7,
             '년': 8,
             '원': 9,
             '투입': 10,
             '여종': 11,
             '데이터': 12,
             '구축': 13,
             '민간': 14,
             '클라우드': 15,
             '외부': 16,
             '연계': 17,
             '체계': 18,
             '개방': 19,
             '강화': 20,
             '데일리': 21,
             '이재운': 22,
             '기자': 23,
             '국가': 24,
             '차원': 25,
             '빅': 26,
             '활용': 27,
             '시대': 28,
             '산업': 29,
             '창출': 30,
             '기존': 31,
             '변화': 32,
             '이르': 33,
             '혁신': 34,
             '장': 35,
             '센터': 36,
             '문': 37,
             '개': 38,
             '분야': 39,
             '경제': 40,
             '발전': 41,

In [107]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
DTM = count_vect.fit_transform(clean_words)
DTM.toarray()

(511, 240)